# Pricing Uncertainty Induced by Climate Change
by [Michael Barnett](https://sites.google.com/site/michaelduglasbarnett/home), [William Brock](https://www.ssc.wisc.edu/~wbrock/) and [Lars Peter Hansen](https://larspeterhansen.org/).

The latest draft of the paper can be found [here](https://larspeterhansen.org/research/papers/).

Notebook by: Jiaming Wang

## Overview

This notebook provides the source code and explanations for how we solve the model setting with __climate damages to growth__. The computational procedures are similar to the model setting with __climate damages to consumption__. Users should refer to the notebook for the [Consumption Damages Model](ConsumptionModel.ipynb) for more computational details.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Overview" data-toc-modified-id="Overview-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Overview</a></span></li><li><span><a href="#Code-and-Illustration" data-toc-modified-id="Code-and-Illustration-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Code and Illustration</a></span><ul class="toc-item"><li><span><a href="#Choosing-key-parameters" data-toc-modified-id="Choosing-key-parameters-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Choosing key parameters</a></span></li><li><span><a href="#Solving-HJB-for-Growth-Damage" data-toc-modified-id="Solving-HJB-for-Growth-Damage-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Solving HJB for Growth Damage</a></span><ul class="toc-item"><li><span><a href="#Remark:--Accounting-for-uncertainty-about-growth-damages" data-toc-modified-id="Remark:--Accounting-for-uncertainty-about-growth-damages-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Remark:  Accounting for uncertainty about growth damages</a></span><ul class="toc-item"><li><span><a href="#Selecting-the-values-of-$\gamma_1,-\gamma_2$" data-toc-modified-id="Selecting-the-values-of-$\gamma_1,-\gamma_2$-2.2.1.1"><span class="toc-item-num">2.2.1.1&nbsp;&nbsp;</span>Selecting the values of $\gamma_1, \gamma_2$</a></span></li><li><span><a href="#Varying-damage-models" data-toc-modified-id="Varying-damage-models-2.2.1.2"><span class="toc-item-num">2.2.1.2&nbsp;&nbsp;</span>Varying damage models</a></span></li><li><span><a href="#Distorted-probabilities" data-toc-modified-id="Distorted-probabilities-2.2.1.3"><span class="toc-item-num">2.2.1.3&nbsp;&nbsp;</span>Distorted probabilities</a></span></li></ul></li><li><span><a href="#Remark-Tolerance-level-and-conergence-criteria-for-HJB" data-toc-modified-id="Remark-Tolerance-level-and-conergence-criteria-for-HJB-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Remark Tolerance level and conergence criteria for HJB</a></span></li></ul></li><li><span><a href="#Simulation" data-toc-modified-id="Simulation-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Simulation</a></span></li><li><span><a href="#SCC-Calculation-Feyman-Kac" data-toc-modified-id="SCC-Calculation-Feyman-Kac-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>SCC Calculation Feyman Kac</a></span><ul class="toc-item"><li><span><a href="#Remark-Convergence-criteria-for-Feyman-Kac" data-toc-modified-id="Remark-Convergence-criteria-for-Feyman-Kac-2.4.1"><span class="toc-item-num">2.4.1&nbsp;&nbsp;</span>Remark Convergence criteria for Feyman Kac</a></span></li></ul></li><li><span><a href="#Probabilities" data-toc-modified-id="Probabilities-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Probabilities</a></span></li></ul></li><li><span><a href="#Results" data-toc-modified-id="Results-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Results</a></span><ul class="toc-item"><li><span><a href="#Worst-case-probabilities-for-9-growth-damage-models" data-toc-modified-id="Worst-case-probabilities-for-9-growth-damage-models-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Worst case probabilities for 9 growth damage models</a></span></li><li><span><a href="#Social-Cost-of-Carbon-Decomposition" data-toc-modified-id="Social-Cost-of-Carbon-Decomposition-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Social Cost of Carbon Decomposition</a></span></li><li><span><a href="#Emission-trajectory" data-toc-modified-id="Emission-trajectory-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Emission trajectory</a></span></li></ul></li></ul></div>

## Code and Illustration

In [ ]:
# Required packages
import os
import sys
sys.path.append('./src')
from supportfunctions import *
sys.stdout.flush()

### Choosing key parameters

In [ ]:
ξp =  1  / 175  # Ambiguity Aversion Paramter 
# We stored solutions for ξp =  1 / 175 to which we referred as "Ambiguity Averse" and ξp = 1000 as “Ambiguity Neutral” in the paper
# Sensible choices are from 0.0002 to 4000, while for parameters input over 0.01 the final results won't alter much

if ξp < 1:
    aversespec = "Averse"
else:
    aversespec = 'Neutral'

v0_guess = None
base_guess = None
worst_guess = None

In [ ]:
# Don't run this cell if you want to solve without smart guess
# Loading Smart guesses
guess = pickle.load(open('./data/Growth{}guess.pickle'.format(aversespec), "rb", -1))
v0_guess = guess['v0']
q_guess = guess['q']
e_guess = guess['e']
base_guess = guess['base']
worst_guess = guess['worst']

### Solving HJB for Growth Damage

In order to solve the HJB corresponding to the planner's problem for the version of the model with climate damages to growth, we use the numerical method as we did for the model version with climate damages to consumption. The main difference comes from the expressions for the coefficients $A(x), B(x), C(x)$, and $D(x)$, which are now given by

\begin{eqnarray}
A(x) &=& - \delta \\
B(x) &=& {\widehat \mu}(x)\\
C(x) &=& {\frac 1 2} \sigma_X(x)'I\sigma_X(x) \\
D(x) &=& \delta (1 - \kappa) \left( \log \left[ {\alpha}  - {\widehat i}(x)  - {\widehat j} (x)  \right] + k  \right) + \delta \kappa \left[ \log {\widehat e}(x)   +  r \right] + {\frac {\xi_m} 2} {\widehat g}(x) \cdot {\widehat g}(x)   + \xi_p {\widehat {\mathbb I}}(x) \\
\end{eqnarray}

where $D(x)$ no longer contains the climate damage impact, but instead ${\widehat \mu}(x)$ incorporates climate damages through its impact on the drift of capital.

#### Remark:  Accounting for uncertainty about growth damages


In calculating growth damamage uncertainty, the term of interest is the contribution of smooth ambiguity to our planner's HJB equation:

\begin{equation}
-\Gamma_i(\beta F)v_{k},
\end{equation}

where the function $\Gamma_i(\beta f)$ is given by

\begin{eqnarray*}
\Gamma_i(\beta F)	=\gamma_{0,i}+\gamma_{1,i}\beta (F+\tilde{F}/\beta)+\frac{\gamma_{2,i}}{2}(\beta (F+\tilde{F}/\beta))^{2}.
\end{eqnarray*}

$\tilde{F}$ is the pre-industrial global mean temperature before climate damages negatively affect growth based on empirical measurements and estimates from \cite{BurkeDavisDiffenbaugh:2018}.


##### Selecting the values of $\gamma_1, \gamma_2$

In this setting, we take a draw of n values of $\gamma = (\gamma_1, \gamma_2)$ based on the distributional structure provided \cite{BurkeDavisDiffenbaugh:2018}:

\begin{eqnarray*}
\gamma \sim \mathcal{N}(\mu, \Sigma), \\
\mu = [\mu_1, \mu_2], \\
\Sigma = \left[ \begin{matrix}
\sigma_1^2 & \rho \\
\rho & \sigma_2^2
\end{matrix} \right].
\end{eqnarray*}

We select points based on the Hermite-Gaussian quadrature abscissas and give probability weights to these outcomes based on the pdf for $\gamma$. 


##### Varying damage models

We must proceed differently for the high damage model with numerical computation.
Form
\begin{equation}
{\rm num}_i(\beta)  = \exp\left( - {\frac 1 \xi_p} \left[ -\Gamma_i(\beta F)v_{k}  \right] \right),
\end{equation}

compute:
\begin{equation}
{\rm scale}_i = \int_\beta {\rm num}_i(\beta) p(\beta) d\beta
\end{equation}

via Gauss-Hermite quadrature, 
and form a density conditioned on model two:
\begin{equation}
{\widetilde q}_i(\beta)  = {\frac {{\rm num}_i(\beta)}{{\rm scale}_i}}
\end{equation}

relative to $p(\beta)$.  

Also compute:
\begin{equation}
{\mathcal  I}_i =  -\xi_p \log {\rm scale}_i,
\end{equation}

along with:
\begin{equation}
{\mathcal J}_i = {\frac {-\Gamma_i(\beta F)v_{k}}{{\rm scale}_i}}
\end{equation}

using Gaussian-Hermite quadrature for the numerator integral. 
Notice that ${\mathcal J}_i$ can be computed for an arbitrary $e$ since the numerator integral does not depend on $e$.
 With these computations in hand, form relative entropy conditioned on model two:
\begin{equation}
{\mathcal R}_i = {\frac 1 {\xi_p}} \left[{\mathcal I}_i - {\mathcal J}_i \right] .
\end{equation}

Notice that ${\mathcal J}_i(e)$ can be computed for an arbitrary $e$ since the numerator integral does not depend on $e$.


##### Distorted probabilities

Worst-case probabilities over models satisfy:
\begin{equation}
{\widetilde \pi}_i \  \ \propto  \  \ {\widehat \pi}_i \exp\left( -{\frac 1 {\xi_p}} {\mathcal I}_i \right)
\end{equation}

where the constant of proportionality scales the right-hand side so that the resulting ${\tilde \pi}_i$ sum to one.


In [ ]:
start_time = time.time()

McD = np.loadtxt('./data/TCRE_MacDougallEtAl2017_update.txt')
par_lambda_McD = McD / 1000

β𝘧 = np.mean(par_lambda_McD)  # Climate sensitivity parameter, MacDougall (2017)
σᵦ = np.var(par_lambda_McD, ddof=1)  # Variance of climate sensitivity parameters
λ = 1.0 / σᵦ 

# Parameters are defined the same as they are in the paper
δ = 0.01        
κ = 0.032       
σ𝘨 = 0.02
σ𝘬 = 0.0161
σ𝘳 = 0.0339 
α = 0.115000000000000
ϕ0 = 0.0600
ϕ1 = 16.666666666666668
μk = -0.034977443912449
ψ0 = 0.112733407891680
ψ1 = 0.142857142857143
F̄ = 13

# False trasient time step
ε = 0.3
# Specifying tolerance level
tol = 1e-12
# Cobweb learning rate
η = 0.05

# Solving HJB
R_min = 0
R_max = 9
F_min = 0
F_max = 750
K_min = 0
K_max = 18


# nR = 30
# nF = 30
# nK = 25

# R = np.linspace(R_min, R_max, nR)
# F = np.linspace(F_min, F_max, nF)
# K = np.linspace(K_min, K_max, nK)

# hR = R[1] - R[0]
# hF = F[1] - F[0]
# hK = K[1] - K[0]

hR = 0.05
hF = 25
hK = 0.15

R = np.arange(R_min, R_max + hR, hR)
nR = len(R)
F = np.arange(F_min, F_max + hF, hF)
nF = len(F)
K = np.arange(K_min, K_max + hK, hK)
nK = len(K)


# Here's how we discretize our state space formulating PDE, see Remark 2 in consumption setting
(R_mat, F_mat, K_mat) = np.meshgrid(R,F,K, indexing = 'ij')
stateSpace = np.hstack([R_mat.reshape(-1,1,order = 'F'),F_mat.reshape(-1,1,order = 'F'),K_mat.reshape(-1,1,order = 'F')])


μ1 = 1.272e-02
μ2 = -4.871e-04
σ1 = 3.248e-03
σ2 = 1.029e-04
ρ12 = -2.859133e-07

σ2 = σ2 * 2
ρ12 = ρ12 * 2

μ = np.matrix([-μ1, -μ2 * 2])
σ = np.matrix([[σ1 ** 2, ρ12], [ρ12, σ2 ** 2]])
Σ = np.matrix([[σᵦ,0,0],[0,σ1 ** 2, ρ12],[0, ρ12, σ2 ** 2]])

[gam1,w1] = quad_points_hermite(3)
gamm1 = np.sqrt(2) * 1 * gam1 + 0
[gam2,w2] = quad_points_hermite(3)
gamm2 = np.sqrt(2) * 1 * gam2 + 0

At = np.linalg.cholesky(σ)
x = np.zeros([2,9])
tmp = [-μ1,-μ2 * 2]
x[:,0] = tmp + At.dot([gamm1[0], gamm2[0]])
x[:,1] = tmp + At.dot([gamm1[0], gamm2[1]])
x[:,2] = tmp + At.dot([gamm1[0], gamm2[2]])
x[:,3] = tmp + At.dot([gamm1[1], gamm2[0]])
x[:,4] = tmp + At.dot([gamm1[1], gamm2[1]])
x[:,5] = tmp + At.dot([gamm1[1], gamm2[2]])
x[:,6] = tmp + At.dot([gamm1[2], gamm2[0]])
x[:,7] = tmp + At.dot([gamm1[2], gamm2[1]])
x[:,8] = tmp + At.dot([gamm1[2], gamm2[2]])

w = np.array([[w1[0],w1[0],w1[0],w1[1],w1[1],w1[1],w1[2],w1[2],w1[2]],
               [w2[0],w2[1],w2[2],w2[0],w2[1],w2[2],w2[0],w2[1],w2[2]]])
gamma1 = x[0,:]
gamma2 = x[1,:]
wgt1 = w[0,:]
wgt2 = w[1,:]

vals = np.linspace(0,30,100)

weight = np.zeros(9)
total_weight = 0
for ite in range(9):
    weight[ite] = wgt1[ite] * wgt2[ite]
    total_weight += weight[ite]
    
weight = weight / total_weight

gamma0 = np.zeros(9)
for ite in range(9):
    gamma0[ite] = max(-(gamma1[ite] * vals + 0.5 * gamma2[ite] * vals ** 2))
    
v0 = κ * R_mat + (1-κ) * K_mat

FC_Err = 1
episode = 0

if v0_guess is not None:
    v0 = v0_guess
    q = q_guess
    e_star = e_guess
    episode = 1
    ε = 0.1


PDE_errs = []
FC_errs = []

while episode == 0 or FC_Err > tol:
    vold = v0.copy()
    # Applying finite difference scheme to the value function
    v0_dr = finiteDiff(v0,0,1,hR) 
    v0_df = finiteDiff(v0,1,1,hF)
    v0_dk = finiteDiff(v0,2,1,hK)

    v0_drr = finiteDiff(v0,0,2,hR)
    v0_drr[v0_dr < 1e-16] = 0
    v0_dr[v0_dr < 1e-16] = 1e-16
    v0_dff = finiteDiff(v0,1,2,hF)
    v0_dkk = finiteDiff(v0,2,2,hK)
    if episode > 2000:
        ε = 0.1
    elif episode > 1000:
        ε = 0.2
    else:
        pass
    
    if episode == 0:
        Acoeff = np.ones(R_mat.shape)
        Bcoeff = ((δ * (1 - κ) * ϕ1 + ϕ0 * ϕ1 * v0_dk) * δ * (1 - κ) / (v0_dr * ψ0 * 0.5) * np.exp(0.5 * (R_mat - K_mat))) / (δ * (1 - κ) * ϕ1)
        Ccoeff = -α  - 1 / ϕ1
        j = ((-Bcoeff + np.sqrt(Bcoeff ** 2 - 4 * Acoeff * Ccoeff)) / (2 * Acoeff)) ** 2
        i = α - j - (δ * (1 - κ)) / (v0_dr * ψ0 * 0.5) * j ** 0.5 * np.exp(0.5 * (R_mat - K_mat))
        q = δ * (1 - κ) / (α - i - j)        
    else:
        e_hat = e_star
        
        # Cobeweb scheme to update controls i and j; q is an intermediary variable that determines i and j
        Converged = 0
        nums = 0
        while Converged == 0:
            i_star = (ϕ0 * ϕ1 * v0_dk / q - 1) / ϕ1
            j_star = (q * np.exp(ψ1 * (R_mat - K_mat)) / (v0_dr * ψ0 * ψ1)) ** (1 / (ψ1 - 1))
            if α > np.max(i_star + j_star):
                q_star = η * δ * (1 - κ) / (α - i_star - j_star) + (1 - η) * q
            else:
                q_star = 2 * q
            if np.max(abs(q - q_star) / η) <= 1e-5:
                Converged = 1
                q = q_star
                i = i_star
                j = j_star
            else:
                q = q_star
                i = i_star
                j = j_star
            
            nums += 1
            
#         if episode % 100 == 0:
        print('Cobweb Passed, iterations: {}, i error: {:10f}, j error: {:10f}'.format(nums, np.max(i - i_star), np.max(j - j_star)))
        
    # calculating growth damage uncertainties, see remark 2.1.1 for details
    a_ = [] 
    b_ = [] 
    c_ = [] 
    λ̃_ = [] 
    β̃_ = [] 
    I_ = [] 
    R_ = [] 
    π̃_ = [] 
    J_ = []

    for ite in range(9):
        a_.append( -v0_dk * (gamma0[ite] + gamma1[ite] * F̄ + 0.5 * gamma2[ite] * F̄ ** 2) )
        b_.append( -v0_dk * F_mat * (gamma1[ite] + gamma2[ite] * F̄) )
        c_.append( -v0_dk * gamma2[ite] * F_mat ** 2 )
        λ̃_.append( λ + c_[ite] / ξp )
        β̃_.append( β𝘧 - c_[ite] / ξp / λ̃_[ite] * β𝘧 - b_[ite] / (ξp * λ̃_[ite]))
        I_.append( a_[ite] - 0.5 * np.log(λ) * ξp + 0.5 * np.log(λ̃_[ite]) * ξp + 0.5 * λ * β𝘧 ** 2 * ξp - 0.5 * λ̃_[ite] * (β̃_[ite]) ** 2 * ξp )
        π̃_.append( weight[ite] * np.exp(-1 / ξp * I_[ite]) )
        J_.append( a_[ite] + b_[ite] * β̃_[ite] + 0.5 * c_[ite] * β̃_[ite] ** 2 + 0.5 * c_[ite] / λ̃_[ite] )
        R_.append((I_[ite] - J_[ite]) / ξp)


    π̃_total = sum(π̃_)
    π̃_norm_ = π̃_ / π̃_total

    B1 = v0_dr - v0_df * np.exp(R_mat)
    C1 = δ * κ
    e = C1 / B1
    e_star = e

    I_term = -1 * ξp * np.log(sum(π̃_))
    drift_distort = sum([x*y for (x,y) in zip(π̃_norm_, J_)])
    RE = sum(x * y + x * np.log(x / z) for (x,y,z) in zip(π̃_norm_, R_, weight))
    RE_total = ξp * RE

    # Formulating coefficients for the HJB False Transient problem, See remark 4 for more details
    A = -δ * np.ones(R_mat.shape)
    B_r = -e_star + ψ0 * (j ** ψ1) * np.exp(ψ1 * (K_mat - R_mat)) - 0.5 * (σ𝘳 ** 2)
    B_k = μk + ϕ0 * np.log(1 + i * ϕ1) - 0.5 * (σ𝘬 ** 2)
    B_f = e_star * np.exp(R_mat)
    C_rr = 0.5 * σ𝘳 ** 2 * np.ones(R_mat.shape)
    C_kk = 0.5 * σ𝘬 ** 2 * np.ones(R_mat.shape)
    C_ff = np.zeros(R_mat.shape)

    D = δ * κ * np.log(e_star) + δ * κ * R_mat + δ * (1 - κ) * (np.log(α - i - j) + K_mat) + I_term #  + drift_distort + RE_total
    
    # Solve linear system using a conjugate-gradient method in C++, see remark 5, 6 for more details
    out = PDESolver(stateSpace, A, B_r, B_f, B_k, C_rr, C_ff, C_kk, D, v0, ε, solverType='False Transient')

    out_comp = out[2].reshape(v0.shape,order = "F")
    
    # Calculating PDE Error and False Transient (lhs) Error
    PDE_rhs = A * v0 + B_r * v0_dr + B_f * v0_df + B_k * v0_dk + C_rr * v0_drr + C_kk * v0_dkk + C_ff * v0_dff + D
    PDE_Err = np.max(abs(PDE_rhs))
    PDE_errs.append(PDE_Err)
    FC_Err = np.max(abs((out_comp - v0)))
    FC_errs.append(FC_Err)
    if episode % 100 == 0:
        print("Episode {:d}: PDE Error: {:.10f}; False Transient Error: {:.10f}; Iterations: {:d}; CG Error: {:.10f}" .format(episode, PDE_Err, FC_Err, out[0], out[1]))
    episode += 1
    v0 = out_comp

print("Episode {:d}: PDE Error: {:.10f}; False Transient Error: {:.10f}; Iterations: {:d}; CG Error: {:.10f}" .format(episode, PDE_Err, FC_Err, out[0], out[1]))
print("--- %s seconds ---" % (time.time() - start_time))


#### Remark Tolerance level and conergence criteria for HJB

Just as we did for the damages to consumption model, we use variable step sizes $\epsilon$ to reduce the time needed for solving the model while ensuring convergence. We use $\epsilon= 0.3$ for the first 1000 iterations, $\epsilon= 0.2$ for iterations 1001-2000, and $\epsilon= 0.1$ thereafter.

The table below lists the convergence criteria and final errors for solving the HJB equations for the different growth damages settings. For solving the model with damages to growth, 


The table below lists the convergence criteria and final errors for solving the HJB equations for the different growth damages settings. For solving the model with damages to growth, we apply a two-stage convergence approach: we first solve the problem at a looser convergence tolerance (CG tolerance at 1e-10 and outer loop tolerance at 1e-8); then we import this solution and solve again using a tighter convergence criteria as listed in the table

| ambiguity    |    stage    | Cobweb tolerance |  outer loop tolerance  | CG tolerance |  PDE Error | dt | iterations | time|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|averse | 1  | 1e-5 | 1e-08 | 1e-10 | 1.55e-5 |variable dt| 10587 | 2.5 hrs
|averse | 2  | 1e-5 | 1e-12 | 1e-14 | 1.55e-5 | 0.1 | 20819(including stage 1) | 2 hrs
|neutral  | 1  | 1e-5 | 1e-08 | 1e-10 | 1.27e-5 | 0.1 | 12855 | 3 hrs
|neutral  | 2  | 1e-5 | 1e-12 | 1e-14 | 1.28e-5 | 0.1 |24857(including stage 1) | 2.5 hrs



### Simulation

In [ ]:
class GridInterp():

    def __init__(self, grids, values, method = 'Linear'):

        # unpacking
        self.grids = grids
        (self.xs, self.ys, self.zs) = grids
        self.nx = len(self.xs)
        self.ny = len(self.ys)
        self.nz = len(self.zs)
        
        self.values = values

        assert (self.nx, self.ny, self.nz) == values.shape, "ValueError: Dimensions not match"
        self.method = method

    def get_value(self, x, y, z):

        if self.method == 'Linear':
            
            func = RegularGridInterpolator(self.grids, self.values)
            return func([x,y,z])[0]

        elif self.method == 'Spline':

            func1 = CubicSpline(self.xs, self.values)
            yzSpace = func1(x)
            
            func2 = CubicSpline(self.ys, yzSpace)
            zSpace = func2(y)
            
            func3 = CubicSpline(self.zs, zSpace)
            return func3(z)

        else:
            raise ValueError('Method Not Supported')

In [ ]:
method = 'Linear'
T = 100
pers = 4 * T
dt = T / pers
nDims = 4
its = 1

gridpoints = (R, F, K)

# emissions
e_func_r = GridInterp(gridpoints, e, method)
def e_func(x):
    return e_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

# investment in reserves
j_func_r = GridInterp(gridpoints, j, method)
def j_func(x):
    return max(j_func_r.get_value(np.log(x[0]), x[2], np.log(x[1])), 0)

# investment in productive capital
i_func_r = GridInterp(gridpoints, i, method)
def i_func(x):
    return i_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

# first derivative of Value function w.r.t R
v_drfunc_r = GridInterp(gridpoints, v0_dr, method)
def v_drfunc(x):
    return v_drfunc_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

# first derivative of Value function w.r.t F
v_dffunc_r = GridInterp(gridpoints, v0_df, method)
def v_dffunc(x):
    return v_dffunc_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

# first derivative of Value function w.r.t k
v_dkfunc_r = GridInterp(gridpoints, v0_dk, method)
def v_dkfunc(x):
    return v_dkfunc_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

# Value function
v_func_r = GridInterp(gridpoints, v0, method)
def v_func(x):
    return v_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

# Distorted model probabilities
π̃_norm_func = []
π̃_norm_func_r = []

for ite in range(len(π̃_norm_)):
    π̃_norm_func_r.append(GridInterp(gridpoints, π̃_norm_[ite], method))
    π̃_norm_func.append(lambda x, ite = ite: π̃_norm_func_r[ite].get_value(np.log(x[0]), x[2], np.log(x[1])))

# Relaive entropies
RE_func_r = GridInterp(gridpoints, RE, method)
def RE_func(x):
    return RE_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

a = []
b = []
c = []
dmg_tilt_ = []
dmg_ = []
base_driftK_r = []
base_driftK = []
tilt_driftK_r = []
tilt_driftK = []

for ite in range(len(π̃_norm_)):
    a.append(gamma0[ite] + gamma1[ite] * F̄ + 0.5 * gamma2[ite] * F̄ ** 2)
    b.append(F_mat * (gamma1[ite] + gamma2[ite] * F̄))
    c.append(gamma2[ite] * F_mat ** 2)
    dmg_tilt_.append(a[ite] + b[ite] * β̃_[ite] + 0.5 * c[ite] * 
                     (β̃_[ite] ** 2) + 0.5 * c[ite] / λ̃_[ite])
    dmg_.append(a[ite] + b[ite] * βf + 0.5 * c[ite] * βf ** 2 + 0.5 * c[ite] / λ)
    # drift distortions
    base_driftK_r.append(GridInterp(gridpoints, dmg_[ite], method))
    base_driftK.append(lambda x, ite = ite: base_driftK_r[ite].get_value(np.log(x[0]), x[2], np.log(x[1])))
    tilt_driftK_r.append(GridInterp(gridpoints, dmg_tilt_[ite], method))
    tilt_driftK.append(lambda x, ite = ite: tilt_driftK_r[ite].get_value(np.log(x[0]), x[2], np.log(x[1])))
    
# Weighted sum of drift distortion
def Gamma_base(x):
    res = 0
    for ite in range(len(weight)):
        res += weight[ite] * base_driftK[ite](x)
    return res

def Gamma_tilted(x):
    res = 0
    for ite in range(len(weight)):
        res += π̃_norm_func[ite](x) * tilt_driftK[ite](x)
    return res

# drifts for each diffusion process
def muR(x):
    return -e_func(x) + ψ0 * (j_func(x) * x[1] / x[0]) ** ψ1
def muK_tilted(x): 
    return (μk + ϕ0 * np.log(1 + i_func(x) * ϕ1)- Gamma_tilted(x))
def muK_base(x): 
    return (μk + ϕ0 * np.log(1 + i_func(x) * ϕ1)- Gamma_base(x))
def muF(x):
    return e_func(x) * x[0]

# volatilities for each diffusion process
def sigmaR(x):
    return np.zeros(x[:4].shape)
def sigmaK(x):
    return np.zeros(x[:4].shape)
def sigmaF(x):
    return np.zeros(x[:4].shape)

# initial points
R_0 = 650
K_0 = 80 / α
F_0 = 870 - 580
initial_val = np.array([R_0, K_0, F_0])

# Set bounds
R_max_sim = np.exp(max(R))
K_max_sim = np.exp(max(K))
F_max_sim = max(F)

R_min_sim = np.exp(min(R))
K_min_sim = np.exp(min(K))
F_min_sim = min(F)

upperbounds = np.array([R_max_sim, K_max_sim, F_max_sim, K_max_sim])
lowerbounds = np.array([R_min_sim, K_min_sim, F_min_sim, K_min_sim])

hists = np.zeros([pers, nDims, its])
e_hists = np.zeros([pers,its])
j_hists = np.zeros([pers,its])
i_hists = np.zeros([pers,its])
RE_hists = np.zeros([pers,its])

pi_tilde_hists = []
for ite in range(len(π̃_norm_)):
    pi_tilde_hists.append(np.zeros([pers,its]))

for iters in range(its):
    hist = np.zeros([pers,nDims])
    e_hist = np.zeros([pers,1])
    i_hist = np.zeros([pers,1])
    j_hist = np.zeros([pers,1])
    
    RE_hist = np.zeros([pers,1])
    pi_tilde_hist = []
    for ite in range(len(π̃_norm_)):
        pi_tilde_hist.append(np.zeros([pers,1]))
    
    hist[0,:] = [R_0, K_0, F_0, K_0]
    e_hist[0] = e_func(hist[0,:]) * hist[0,0]
    i_hist[0] = i_func(hist[0,:]) * hist[0,1]
    j_hist[0] = j_func(hist[0,:]) * hist[0,0]
    RE_hist[0] = RE_func(hist[0,:])
    
    for ite in range(len(π̃_norm_)):
        pi_tilde_hist[ite][0] = π̃_norm_func[ite](hist[0,:])
    
    for tm in range(1,pers):
        shock = norm.rvs(0,np.sqrt(dt),nDims)
        hist[tm,0] = cap(hist[tm-1,0] * np.exp((muR(hist[tm-1,:])- 0.5 * sum((sigmaR(hist[tm-1,:])) ** 2))* dt + sigmaR(hist[tm-1,:]).dot(shock)),lowerbounds[0], upperbounds[0])
        hist[tm,1] = cap(hist[tm-1,1] * np.exp((muK_base(hist[tm-1,:])- 0.5 * sum((sigmaK(hist[tm-1,:])) ** 2))* dt + sigmaK(hist[tm-1,:]).dot(shock)),lowerbounds[1], upperbounds[1])
        hist[tm,2] = cap(hist[tm-1,2] + muF(hist[tm-1,:]) * dt + sigmaF(hist[tm-1,:]).dot(shock), lowerbounds[2], upperbounds[2])
        hist[tm,3] = cap(hist[tm-1,3] * np.exp((muK_tilted(hist[tm-1,:])- 0.5 * sum((sigmaK(hist[tm-1,:])) ** 2))* dt + sigmaK(hist[tm-1,:]).dot(shock)),lowerbounds[3], upperbounds[3])

        e_hist[tm] = e_func(hist[tm-1,:]) * hist[tm-1,0]
        i_hist[tm] = i_func(hist[tm-1,:]) * hist[tm-1,1]
        j_hist[tm] = j_func(hist[tm-1,:]) * hist[tm-1,1]
        RE_hist[tm] = RE_func(hist[tm-1, :])
        
        for ite in range(len(π̃_norm_)):
            pi_tilde_hist[ite][tm] = π̃_norm_func[ite](hist[tm-1,:])

    hists[:,:,iters] = hist
    e_hists[:,[iters]] = e_hist
    i_hists[:,[iters]] = i_hist
    j_hists[:,[iters]] = j_hist
    
    RE_hists[:,[iters]] =  RE_hist
    
    for ite in range(len(π̃_norm_)):
        pi_tilde_hists[ite][:,[iters]] = pi_tilde_hist[ite]

### SCC Calculation Feyman Kac

The growth damages model setting is solved using the same numerical method as the consumption damage model, with a few minor adjustments made to the equations and calculations. First, the expressions for $\Psi^*(x)$ and $\bar{\Psi}(x)$ are now given by 

\begin{align*}
 \Psi^*(x) & = -  \int_{\Theta} V_k(x) \left[ \nabla \Gamma(\beta f) \beta \right] q^*(\theta | x) P(d\theta) \\
 \bar{\Psi}(x) & = -  \int_{\Theta} V_k(x) \left[ \nabla \Gamma(\beta f) \beta \right] P(d\theta).
\end{align*}

Second, these integral expressions can be calculated using entirely analytical simplifictions as will be outlined below.

Finally, there are no longer direct contributions to the damage evolution that have integral expressions that need to be numerically calculated and added to the PDEs derived from application of the F-K formula.

With these differences we can apply the analogous approach to the consumption damage setting to derive $ecc^*$, $\bar{ecc}$, and $ucc^*$ for the growth damages model setting. As before, after imposing the same finite difference scheme the PDEs are characterized by unconditionally linear equations that we can solve directly by approximately inverting the linear systems using the conjugate gradient method without the use of the false transient or other iterative schemes. These PDEs can also be alternatively be solved using the implicit, finite-difference scheme with the conjugate gradient solver augmented by a false transient where the coefficients are definied as in the consumption damage case, appropriately adjusted for the $\mu^*_X[x,a^*(x)]$, $\mu_X[x,a^*(x)]$, $\Psi^*(x)$, and $\bar{\Psi}(x)$ that correspond to the growth damages model.

In [ ]:
a, b, c, dmg_ = ([] for ite in range(4)) 
for ite in range(len(π̃_norm_)):
    a.append(np.zeros(F_mat.shape) )
    b.append(v0_dk * (gamma1[ite] + gamma2[ite] * F̄))
    c.append(2 * v0_dk * gamma2[ite] * F_mat)
    dmg_.append(a[ite] + b[ite] * βf + 0.5 * c[ite] * βf ** 2 + 0.5 * c[ite] / λ)
    
flow_base = sum(w * d for w, d in zip(weight, dmg_))

a, b, c, dmg_ = ([] for ite in range(4)) 
for ite in range(len(π̃_norm_)):
    a.append(v0_dk * (gamma0[ite] + gamma1[ite] * F̄ + 0.5 * gamma2[ite] * F̄ ** 2))
    b.append(v0_dk * F_mat * (gamma1[ite] + gamma2[ite] * F̄))
    c.append(v0_dk * gamma2[ite] * F_mat ** 2)
    dmg_.append(a[ite] + b[ite] * βf + 0.5 * c[ite] * βf ** 2 + 0.5 * c[ite] / λ)

Gamma_base = sum(w * d for w, d in zip(weight, dmg_))

A = -δ * np.ones(R_mat.shape)
# B_r = -e + ψ0 * (j ** ψ1) - 0.5 * (σ𝘳 ** 2)
B_r = -e + ψ0 * (j ** ψ1) * np.exp(ψ1 * (K_mat - R_mat)) - 0.5 * (σ𝘳 ** 2)
B_k = μk + ϕ0 * np.log(1 + i * ϕ1) - 0.5 * (σ𝘬 ** 2) - Gamma_base
B_f = e * np.exp(R_mat)
C_rr = 0.5 * σ𝘳 ** 2 * np.ones(R_mat.shape)
C_kk = 0.5 * σ𝘬 ** 2 * np.ones(R_mat.shape)
C_ff = np.zeros(R_mat.shape)
D = flow_base

if base_guess is None:
    base_guess = np.zeros(R_mat.shape)


# out = PDESolver(stateSpace, A, B_r, B_f, B_k, C_rr, C_ff, C_kk, D, v0, ε, 'Feyman Kac')
out = PDESolver(stateSpace, A, B_r, B_f, B_k, C_rr, C_ff, C_kk, D, base_guess, ε, solverType='Feyman Kac')
v0_base = out[2].reshape(v0.shape, order="F")
v0_base = v0_base

v0_dr_base = finiteDiff(v0_base,0,1,hR,1e-16) 
v0_df_base = finiteDiff(v0_base,1,1,hF)
v0_dk_base = finiteDiff(v0_base,2,1,hK)

v0_drr_base = finiteDiff(v0_base,0,2,hR)
v0_dff_base = finiteDiff(v0_base,1,2,hF)
v0_dkk_base = finiteDiff(v0_base,2,2,hK)


PDE_rhs = A * v0_base + B_r * v0_dr_base + B_f * v0_df_base + B_k * v0_dk_base + C_rr * v0_drr_base + C_kk * v0_dkk_base + C_ff * v0_dff_base + D
PDE_Err = np.max(abs(PDE_rhs))
print("Feyman Kac Base Model Solved. PDE Error: {:.10f}; Iterations: {:d}; CG Error: {:.10f}".format(PDE_Err, out[0], out[1]))

a, b, c, dmg_tilt_ = ([] for ite in range(4)) 
for ite in range(len(π̃_norm_)):
    a.append(np.zeros(F_mat.shape) )
    b.append(v0_dk * (gamma1[ite] + gamma2[ite] * F̄))
    c.append(2 * v0_dk * gamma2[ite] * F_mat)
    dmg_tilt_.append(a[ite] + b[ite] * β̃_[ite] + 0.5 * c[ite] * β̃_[ite] ** 2 + 0.5 * c[ite] / λ̃_[ite])
    
flow_tilted = sum(w * d for w, d in zip(π̃_norm_, dmg_tilt_))

a, b, c, dmg_tilt_ = ([] for ite in range(4)) 
for ite in range(len(π̃_norm_)):
    a.append(v0_dk * (gamma0[ite] + gamma1[ite] * F̄ + 0.5 * gamma2[ite] * F̄ ** 2))
    b.append(v0_dk * F_mat * (gamma1[ite] + gamma2[ite] * F̄))
    c.append(v0_dk * gamma2[ite] * F_mat ** 2)
    dmg_tilt_.append(a[ite] + b[ite] * β̃_[ite] + 0.5 * c[ite] * β̃_[ite] ** 2 + 0.5 * c[ite] / λ̃_[ite])

Gamma_tilted = sum(w * d for w, d in zip(π̃_norm_, dmg_tilt_))

A = -δ * np.ones(R_mat.shape)
# B_r = -e + ψ0 * (j ** ψ1) - 0.5 * (σ𝘳 ** 2)
B_r = -e + ψ0 * (j ** ψ1) * np.exp(ψ1 * (K_mat - R_mat)) - 0.5 * (σ𝘳 ** 2)
B_k = μk + ϕ0 * np.log(1 + i * ϕ1) - 0.5 * (σ𝘬 ** 2) - Gamma_tilted
B_f = e * np.exp(R_mat)
C_rr = 0.5 * σ𝘳 ** 2 * np.ones(R_mat.shape)
C_kk = 0.5 * σ𝘬 ** 2 * np.ones(R_mat.shape)
C_ff = np.zeros(R_mat.shape)
D = flow_tilted

if worst_guess is None:
    worst_guess = np.zeros(R_mat.shape)

# out = PDESolver(stateSpace, A, B_r, B_f, B_k, C_rr, C_ff, C_kk, D, v0, ε, 'Feyman Kac')
out = PDESolver(stateSpace, A, B_r, B_f, B_k, C_rr, C_ff, C_kk, D, worst_guess, ε, solverType='Feyman Kac')
v0_worst = out[2].reshape(v0.shape, order="F")
v0_worst = v0_worst

v0_dr_worst = finiteDiff(v0_worst,0,1,hR,1e-16) 
v0_df_worst = finiteDiff(v0_worst,1,1,hF)
v0_dk_worst = finiteDiff(v0_worst,2,1,hK)

v0_drr_worst = finiteDiff(v0_worst,0,2,hR)
v0_dff_worst = finiteDiff(v0_worst,1,2,hF)
v0_dkk_worst = finiteDiff(v0_worst,2,2,hK)

PDE_rhs = A * v0_worst + B_r * v0_dr_worst + B_f * v0_df_worst + B_k * v0_dk_worst + C_rr * v0_drr_worst + C_kk * v0_dkk_worst + C_ff * v0_dff_worst + D
PDE_Err = np.max(abs(PDE_rhs))
print("Feyman Kac Worst Model Solved. PDE Error: {:.10f}; Iterations: {:d}; CG Error: {:.10f}".format(PDE_Err, out[0], out[1]))

In [ ]:
# Total SCC
MC = δ * (1-κ) / (α * np.exp(K_mat) - i * np.exp(K_mat) - j * np.exp(K_mat))
ME = δ * κ / (e * np.exp(R_mat))
SCC = 1000 * ME / MC
SCC_func_r = GridInterp(gridpoints, SCC, method)

def SCC_func(x): 
#     return SCC_func_r.get_value(np.log(x[0]), x[2], np.log(x[3]))
    return SCC_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

# Private SCC
ME1 = v0_dr * np.exp(-R_mat)
SCC1 = 1000 * ME1 / MC
SCC1_func_r = GridInterp(gridpoints, SCC1, method)
def SCC1_func(x):
#     return SCC1_func_r.get_value(np.log(x[0]), x[2], np.log(x[3]))
    return SCC1_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

# Feyman Kac solution under original unadjusted probabilities
ME2_base = v0_base
SCC2_base = 1000 * ME2_base / MC
SCC2_base_func_r = GridInterp(gridpoints, SCC2_base, method)
def SCC2_base_func(x):
#     return SCC2_base_func_r.get_value(np.log(x[0]), x[2], np.log(x[3]))
    return SCC2_base_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))


ME2_base_a = -v0_df
SCC2_base_a = 1000 * ME2_base_a / MC
SCC2_base_a_func_r = GridInterp(gridpoints, SCC2_base_a, method)
def SCC2_base_a_func(x):
#     return SCC2_base_a_func_r.get_value(np.log(x[0]), x[2], np.log(x[3]))
    return SCC2_base_a_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

# Feyman Kac solution under ambiguity-adjusted probability
ME2_tilt = v0_worst
SCC2_tilt = 1000 * ME2_tilt / MC
SCC2_tilt_func_r = GridInterp(gridpoints, SCC2_tilt, method)
def SCC2_tilt_func(x):
#     return SCC2_tilt_func_r.get_value(np.log(x[0]), x[2], np.log(x[3]))
    return SCC2_tilt_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

# Interpolating via simulated trajectories 
SCC_values = np.zeros([pers,its])
SCC1_values = np.zeros([pers,its])
SCC2_base_values = np.zeros([pers,its])
SCC2_tilt_values = np.zeros([pers,its])
SCC2_base_a_values = np.zeros([pers,its])

for tm in range(pers):   # time horizons
    for path in range(its):    # Number of trajectories, currently it's 1
        SCC_values[tm, path] = SCC_func(hists[tm,:,path])
        SCC1_values[tm, path] = SCC1_func(hists[tm,:,path])
        SCC2_base_values[tm, path] = SCC2_base_func(hists[tm,:,path]) 
        SCC2_tilt_values[tm, path] = SCC2_tilt_func(hists[tm,:,path])
        SCC2_base_a_values[tm, path] = SCC2_base_a_func(hists[tm,:,path])
        
SCC_total = np.mean(SCC_values,axis = 1)
SCC_private = np.mean(SCC1_values,axis = 1)
SCC2_FK_base = np.mean(SCC2_base_values,axis = 1)
SCC2_FK_tilt = np.mean(SCC2_tilt_values,axis = 1)

uncertain = SCC2_FK_tilt - SCC2_FK_base

uncertain = SCC2_FK_tilt - SCC2_FK_base

SCCs = {}
SCCs['SCC'] = SCC_total   # Total SCC
SCCs['SCC1'] = SCC_private   # private SCC
SCCs['SCC2'] = SCC2_FK_base   # external SCC
SCCs['SCC3'] = SCC2_FK_tilt - SCC2_FK_base  # Uncertainty SCC

#### Remark Convergence criteria for Feyman Kac

Below table listed the convergence criteria and final errors for solving HJB equations at different growth damages setting which we used in the paper. As the discretized state space contains over 3 million points, even the Conjugate Gradient approximation is quite slow to converge to a small error. Therefore, we cap the max iterations for the conjugate gradient solver to be 400,000.

| ambiguity    |   damage specification   | iterations | CG Error |  PDE Error | Approx time
|:-:|:-:|:-:|:-:|:-:|:-:|
|averse | base  | 340,113 | 1e-14 | 4.50e-6|2 hrs
|averse | worst | 391,635 | 1e-14| 2.21e-6| 2.5 hrs
|neutral | base  | 577,167 | 1e-14 | 2.21e-6|3.5 hrs
|neutral | worst | 556,956 | 1e-14 | 2.21e-6|3.5 hrs

### Probabilities

Here we caculated and plotted the ${\widetilde q}_2(\beta)$ implied Ambiguity-Adjusted Probabilities for 9 growth damage modesls 

In [ ]:
Dists = {}

a = β𝘧 - 5 * np.sqrt(σᵦ)
b = β𝘧 + 5 * np.sqrt(σᵦ)
a_10std = β𝘧 - 10 * np.sqrt(σᵦ)
b_10std = β𝘧 + 10 * np.sqrt(σᵦ)
beta_f_space = np.linspace(a_10std,b_10std,200)

original_dist = norm.pdf(beta_f_space, β𝘧, np.sqrt(σᵦ))
Dists['Original'] = original_dist

R_value = np.mean(hists[:,0,:], axis = 1)
K_value = np.mean(hists[:,1,:], axis = 1)
F_value = np.mean(hists[:,2,:], axis = 1)

R_func_r = []
R_func = []
β̃_func_r = []
β̃_func = []
λ̃_func_r = []
λ̃_func = []
for ite in range(len(R_)):
    R_func_r.append(GridInterp(gridpoints, R_[ite], method))
    R_func.append(lambda x, ite = ite: R_func_r[ite].get_value(np.log(x[0]), x[2], np.log(x[1])))
    β̃_func_r.append(GridInterp(gridpoints, β̃_[ite], method))
    β̃_func.append(lambda x, ite = ite: β̃_func_r[ite].get_value(np.log(x[0]), x[2], np.log(x[1])))
    λ̃_func_r.append(GridInterp(gridpoints, λ̃_[ite], method))
    λ̃_func.append(lambda x, ite = ite: λ̃_func_r[ite].get_value(np.log(x[0]), x[2], np.log(x[1])))

hists_mean = np.mean(hists, axis = 2)
RE_plot = np.zeros(pers)
weight_plot = [np.zeros([pers,1]) for ite in range(len(π̃_norm_func))]
for tm in range(pers):
    RE_plot[tm] = RE_func(hists_mean[tm,:])
    for ite in range(len(π̃_norm_func)):
        weight_plot[ite][tm] = π̃_norm_func[ite](hists_mean[tm,:])

for tm in [1,100,200,300,400]:
    R0 = hists[tm-1,0,0]
    K0 = hists[tm-1,1,0]
    F0 = hists[tm-1,2,0]
    weights_prob = []
    mean_distort_ = []
    lambda_tilde_ = []
    tilt_dist_ = []
    
    for ite in range(len(π̃_norm_func)):
        weights_prob.append(π̃_norm_func[ite]([R0, K0, F0]))
        mean_distort_.append(β̃_func[ite]([R0, K0, F0]) - βf)
        lambda_tilde_.append(λ̃_func[ite]([R0, K0, F0]))
        tilt_dist_.append(norm.pdf(beta_f_space, mean_distort_[ite] + βf, 1 / np.sqrt(lambda_tilde_[ite])))
        
    weighted = sum(w * til for w, til in zip(weights_prob, tilt_dist_) )
    Dists['Year' + str(int((tm) / 4))] = dict(tilt_dist = tilt_dist_, weighted = weighted, weights = weights_prob)

## Results

### Worst case probabilities for 9 growth damage models

In [ ]:
growthdensityPlot(beta_f_space, Dists)

### Social Cost of Carbon Decomposition

In [ ]:
growthSCCDecomposePlot(SCCs, ξp)

### Emission trajectory

In [ ]:
growthemissionPlot(ξp, e_hists)